## Fetch and prepare FWF data

### 1. Fetch

Fetch PulseWaves data of a flight line from the NYU Spatial Data Repo.

In [1]:
project='d15'
line='115601'

# fetch pulsewaves
!../scripts/fetch.sh $project $line fwf-pw

--2020-07-31 19:31:57--  https://archive.nyu.edu/retrieve/79995/nyu_2451_38625_fwf_plswvs_F_150326_115601.zip
Resolving archive.nyu.edu (archive.nyu.edu)... 128.122.108.142
Connecting to archive.nyu.edu (archive.nyu.edu)|128.122.108.142|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 961431605 (917M) [application/octet-stream]
Saving to: ‘tmp1596220317.27054/nyu_2451_38625_fwf_plswvs_F_150326_115601.zip’

nyu_2451_38625_fwf_ 100%[===================>] 916.89M  11.8MB/s    in 77s     

2020-07-31 19:33:15 (11.8 MB/s) - ‘tmp1596220317.27054/nyu_2451_38625_fwf_plswvs_F_150326_115601.zip’ saved [961431605/961431605]

Archive:  tmp1596220317.27054/nyu_2451_38625_fwf_plswvs_F_150326_115601.zip
   creating: tmp1596220317.27054/nyu_2451_38625_pulse/
  inflating: tmp1596220317.27054/nyu_2451_38625_pulse/F_150326_115601.pls  
  inflating: tmp1596220317.27054/nyu_2451_38625_pulse/F_150326_115601.wvs  
moving tmp159622031

Check that the PulseWaves files have been downloaded and unpacked

In [2]:
!ls ../data/fwf-pw/*

../data/fwf-pw/10552_NYU_M2_Pulse_Waves_MTA_Resolved-Scanner1-190511_161348_1-originalpoints.pls
../data/fwf-pw/10552_NYU_M2_Pulse_Waves_MTA_Resolved-Scanner1-190511_161348_1-originalpoints.wvs
../data/fwf-pw/F_150326_115601.pls
../data/fwf-pw/F_150326_115601.wvs


Display the Pulse file header

In [3]:
pls_files = !ls ../data/fwf-pw/*.pls
first_pls_file = pls_files[0]
!java -jar "../jar/umg-cli-0.2.0-SNAPSHOT-jar-with-dependencies.jar" pulseinfo -i $first_pls_file

--------------------------------------------------
PULSE HEADER
--------------------------------------------------
fileSignature                  PulseWavesPulse
globalParameter                00000000
fileSourceID                   0
guidData1                      1900857167
guidData2                      41870
guidData3                      17123
guidData4                      
systemIdentifier               RiPROCESS 1.8.5
generatingSoftware             PulseWaves DLL 0.3 r11 (150617) by rapidlasso
fileCreationDate               149
fileCreationYear               2019
versionMajor                   0
versionMinor                   3
headerSize                     352
offsetToPulseData              9338
numberOfPulses                 4808941
pulseFormat                    0
pulseAttributes                0
pulseSize                      48
pulseCompression               0
reserved                       0
numberOfVariableLengthRecords  18
numberOfAppededVariableLengthRecords 0
tScaleF

### 2. Convert the data to PWMsg format

The Java module below pairs each pulse with its waveforms and encodes the data in a [Protocol Buffers](https://developers.google.com/protocol-buffers) format. The encoded data (i.e. PWMsg) are segmented to small chunks (e.g. 100000 pulses) to make them more managable. The format protocol is in the `data/protobuf/pulsewaves.proto` file.

In [4]:
!mkdir -p ../data/pwmsg
!java -jar "../jar/umg-cli-0.2.0-SNAPSHOT-jar-with-dependencies.jar" pw2proto -idir ../data/fwf-pw -odir ../data/pwmsg -meta_file ../data/metadata/$project-meta.json -segment 100000 -subseq 0 5000

2020-07-31 19:33:41,139 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - Offset: [316000.000, 234000.000, 0.000, , 0.000]
2020-07-31 19:33:41,140 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - Scale: [0.001, 0.001, 0.001, , 0.000]
2020-07-31 19:33:41,140 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - Flight ID map contains 41 entries.
2020-07-31 19:33:41,143 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - File source ID: 1
2020-07-31 19:33:41,145 [pool-1-thread-1] INFO  umg.core.lidar.protobuf.cli.PW2Proto - Processing /home/vvo/code/vvogit/sunsetpark/notebooks/../data/fwf-pw/F_150326_115601.pls
Processing /home/vvo/code/vvogit/sunsetpark/notebooks/../data/fwf-pw/F_150326_115601.pls
2020-07-31 19:33:41,145 [main] DEBUG umg.core.lidar.protobuf.cli.PW2Proto - File source ID: 0
2020-07-31 19:33:41,145 [pool-1-thread-2] INFO  umg.core.lidar.protobuf.cli.PW2Proto - Processing /home/vvo/code/vvogit/sunsetpark/notebooks/../data/fwf-pw/10552_NYU_M2_Pulse_Waves_MTA_Resolved-Sca

Check the PWMsg data 

In [5]:
!ls ../data/pwmsg

10552_NYU_M2_Pulse_Waves_MTA_Resolved-Scanner1-190511_161348_1-originalpoints-0.pwmsg
F_150326_115601-0.pwmsg


Check notebook [01_pwmsg-d15.ipynb](01_pwmsg-d15.ipynb) for how to work with the PWMsg files using Python